## 1. MalConv 만들기

* 필요한 Package 다운로드

In [ ]:
!pip install torch
!pip install numpy

* MalConv Class를 만들기 위한 package 로드

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

* MalConv Class 만들기

In [ ]:
class MalConv(nn.Module):
    def __init__(self, out_size=2, channels=128, window_size=512, embd_size=8):
        
        ### MalConv 제작을 위한 variable 초기화 ###
        # 1. nn.Module 상속 선언
        # 2. Embedding Vector 구성(257(0~256) * embd_size)
        # 3. window_size 선언
        # 4. 2개 1D Convolution Layer 선언(embd_size * channels * window_size)
        # 5. MaxPooling Layer 선언
        # 6. Fully Connected Layer 선언(channels * channels)
        # 7. Fully Connected Layer 선언(channels * out_size)

        super(MalConv, self).__init__()
        self.embd = nn.Embedding(257, embd_size, padding_idx=0)        
        self.window_size = window_size
        self.conv_1 = nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)
        self.conv_2 = nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)        
        self.pooling = nn.AdaptiveMaxPool1d(1)        
        self.fc_1 = nn.Linear(channels, channels)
        self.fc_2 = nn.Linear(channels, out_size)
    
    def forward(self, x):

        ### MalConv 학습과정 구축 ###
        # 1. input x에 대한 embedding vector 구축
        # 2. CNN 연산을 위한 차원 교환
        # 3. conv_1을 이용한 cnn값 추출
        # 4. conv_2 및 sigmoid를 이용한 값 추출
        # 5. 3,4번의 추출된 값의 합성곱 연산 수행
        # 6. 5번 값에 대한 MaxPooling 수행
        # 7. FC layer 연산을 위한 shape 변경
        # 8. fc_1을 통한 FC layer 연산 및 Relu 활성화 함수 연산
        # 9. 8의 결과를 fc_2를 통한 FC layer 연산 수행
        # 10. 최종 결과 return
        
        x = self.embd(x.long())
        x = torch.transpose(x,-1,-2)
        cnn_value = self.conv_1(x)
        gating_weight = torch.sigmoid(self.conv_2(x))        
        x = cnn_value * gating_weight        
        x = self.pooling(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)        
        return x

## 2. MalConv 모델 이용하기

### 1. MalConv 모델 이용 준비
* 앞서 구축한 MalConv class 불러오기

In [ ]:
# 256 channel, 512 window_size, embed_size 8의 Malconv 모델 구성
malconv_model = MalConv(channels=256, window_size=512, embed_size=8)

* 기 구축한 문서형 Stream data의 MalConv 모델 가중치 불러오기 및 MalConv 적용

In [ ]:
weight = torch.load("./malRNN_doc.pt")
malconv_model.load_state_dict(weight)

### 2. MalConv를 이용한 문서형 stream data의 악성 여부 확인

In [ ]:
data1 = []
data2 = []

* 악성 탐지를 위한 과정 구축

In [ ]:
def detect(malconv, stream_data):
    
    ### MalConv를 이용한 Stream data 악성 여부 확인 함수 ###
    # malconv: 앞서 구축한 MalConv 모델
    # stream_data : 악성 여부를 확인할 stream data

    # 1. stream data를 malconv에 맞는 형식으로 변환
    # 2. malconv를 통해 값 도출
    # 3. 도출된 값을 Softmax 함수를 이용한 확률값으로 변환

    stream_data = torch.from_numpy(np.fromduffer(stream_data, dtype=np.uint8)[np.newaxis, :])
    output = malconv(stream_data)
    output = F.softmax(malconv(output), dim=-1)
    return output